# 36610 - Python and Unix for Bioinformaticians

## Project 6: Resistance to antibiotics

Details on how to read compressed files can be found [here](https://docs.python.org/3/library/gzip.html#gzip.decompress).

In [11]:
import gzip # import gzip to decompress .gz files

resistance_genes = open("data/resistance_genes_sub.fsa", "r")
reads1 = gzip.open("data/Unknown3_raw_reads_1.txt.gz", "rt") # 'rt' -> read-in in text mode
reads2 = gzip.open("data/Unknown3_raw_reads_2.txt.gz", "rt")

In [3]:
# Open subsets to increase speed
ResFile = open("data/resistance_genes_sub.fsa", "r")
reads1 = open("data/Unknown3_raw_reads_1_sub.txt", "r")
outfile = open("data/results.txt", "w")

# define variables
header  = ""   # will contain the header for each FASTA sequence
sub_seq = ""   # i think this variable is not necessarily needed – used to store part of the FASTA sequence
seq     = []   # used to store entire FASTA sequence
line_count = 3 # 

# k-mer length
def kmer_generator(sequence, kmer_length):
    kmer_list = list()
    for i in range(0, len(sequence), 1):
        if (i < len(sequence)-kmer_length+1):
            kmer_list.append(sequence[i:i+kmer_length])
    return(kmer_list)



### resistance gene
    ### - k-mers
### -- read (use in; NO k-mer)
    ### for each k-mer in read
        ### count

for line in ResFile:
    line = line.rstrip()
    if line.startswith(">"):
        header = line
        if sub_seq:
            seq.append(sub_seq)
            seq = "".join(seq)
            #print(header + "\n" + seq) # header + full sequence
            # generate k-mers right here
            res_kmer = kmer_generator(seq, 19) # k-mers of the resistence gene
            print(header, file=outfile)
            #print(res_kmer)
            res_kmer_count = get_counter_list(res_kmer)
            #print(res_kmer_count)
            
            for line in reads1:
                line = line.rstrip()
                if (line_count % 4 == 0): # prints every 4th line
                    sequence = line
                    # if res k-mer in read
                    
                    #read_kmer = kmer_generator(sequence, 19)
                    #incr_counter_list(res_kmer_count, read_kmer)
                line_count += 1
            
            for pos in range(len(res_kmer_count)):
                print(res_kmer_count[pos][0], res_kmer_count[pos][1], file=outfile)
            seq = []
    else:
        sub_seq = line
        seq.append(sub_seq)
# the following is to process the last FASTA entry
seq.append(sub_seq)
seq = "".join(seq)
#print(header + "\n" + seq)
# generate k-mers right here
res_kmer = kmer_generator(seq, 19) # k-mers of the resistence gene
#print(header)
#print(res_kmer)

ResFile.close()
reads1.close()

In [1]:
RES_TEST  = ["ABC", "DEF", "GHI", "ABC"]
READ_LIST_2 = ["DEF", "GHI"]

def get_counter_list(RES_LIST):
    counterlist = list()
    for entry in RES_LIST:
        counterlist.append([entry, 0])
    return(counterlist)

# counterlist = get_counter_list(RES_TEST) # print list with k-mer and its coverage

def incr_counter_list(RES_LIST, READ_LIST):
    for pos in range(len(RES_LIST)):
        for read_kmer in READ_LIST:
            if read_kmer == RES_LIST[pos][0]: # in
                #print("DOM")
                RES_LIST[pos][1] += 1
        #print(RES_LIST[pos][0], RES_LIST[pos][1])
    #return(RES_LIST) # do not return to modify original list
    
incr_counter_list(get_counter_list(RES_TEST), READ_LIST_2)

In [76]:
reads1 = open("data/Unknown3_raw_reads_1_sub.txt")

#sequence = ""
#line_count = 3

#def get_read1(x):
for line in reads1:
    line = line.rstrip()
    if (line_count % 4 == 0): # prints every 4th line
        sequence = line
        #print(sequence)
    line_count += 1
    
# same s#*t
def get_read1(line):
    line_count = 3
    line = line.rstrip()
    if (line_count % 4 == 0): # prints every 4th line
        sequence = line
        return(sequence)
    line_count += 1

reads1.close()

In [3]:
# k_mer of resistance genes
def k_mer(seq, k_mer_length):
    k_mer_list = list()
    for i in range(0, len(seq), 1):
        if (i < len(seq) - k_mer_length + 1):
            k_mer_list.append(sequence[i:i+k_mer_length])
    return(k_mer_list)

In [ ]:
RES  = "ACGTACGTACGT" # Resistence gene
SEQ1 = "TTACGT"       # Contains last 3 resistance bases (so only part)
SEQ2 = "TACGTACGT"    # Entire sequence part of resistence gene
SEQ3 = "TACGTTA"      # Contains first 4 resistence bases (first part)
# Initialize list that keeps track of the depth
seq_list_count = list()

for i in range(len(RES)):
    seq_list_count.append(0)

In [47]:
# k-mer length
def kmer_generator(sequence, kmer_length):
    kmer_list = list()
    for i in range(0, len(sequence), 1):
        if (i < len(sequence)-kmer_length+1):
            kmer_list.append(sequence[i:i+kmer_length])
    return(kmer_list)

kmer_length = 3

SEQ1_kmer = kmer_generator(SEQ1, kmer_length)
SEQ2_kmer = kmer_generator(SEQ2, kmer_length)
SEQ3_kmer = kmer_generator(SEQ3, kmer_length)
RES_kmer  = kmer_generator(RES, kmer_length)

# check first SEQ k-mer (idea: COMPLETE SEQ-RES MATCH)
# for this method only the first k_mer of SEQ needs to be generated => shorter runtime
def first_kmer_check(SEQ_kmers, RES_kmers):
    for i in range(0, len(RES_kmers)-1, 1):
        if (SEQ_to_check[0] == RES_to_check[i]):
            print("Found match in first iteration. Investigating this SEQ sequence further.")
            return(True, i)
    print("No initial match. Discarding this SEQ.")
    return(False, 0)

SEQ_to_check = SEQ3_kmer
RES_to_check = RES_kmer

first_hit, start_pos = first_kmer_check(SEQ_to_check, RES_to_check)
end_pos = 0

i = 0
j = start_pos

length_SEQ = len(SEQ_to_check) - 1
length_RES = len(RES_to_check) - 1

is_mismatch = False

if (first_hit):
    while (i <= length_SEQ):
        while (j <= length_RES):
            print(i, SEQ_to_check[i], j, RES_to_check[j])
            if (SEQ_to_check[i] == RES_to_check[j]):
                print("Match")
                end_pos = j
                j += 1
                break
            elif (SEQ_to_check[i] != RES_to_check[j]):
                print("Mismatch")
                is_mismatch = True
                break
            j += 1
        if (is_mismatch):
            j = 0
            break
        i += 1

print("Start in RES:", start_pos, "End in RES:", start_pos+end_pos)

for i in range(start_pos, start_pos+end_pos, 1):
    seq_list_count[i] += 1

for i in range(len(RES)):
    print(RES[i], seq_list_count[i])

# RES:  ACGTACGTACGT
# SEQ2: ---TACGTTA--

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Found match in first iteration. Investigating this SEQ sequence further.
0 TAC 3 TAC
Match
1 ACG 4 ACG
Match
2 CGT 5 CGT
Match
3 GTT 6 GTA
Mismatch
Start in RES: 3 End in RES: 8
A 0
C 0
G 0
T 1
A 1
C 1
G 1
T 1
A 0
C 0
G 0
T 0


In [2]:
# print first lines of compressed FASTQ file
count = 0
for line in reads1:
    if (count < 15):
        print(line.rstrip())
        count = count + 1
    else:
        break

@ILLUMINA-3BDE4F_0027:2:1:9216:1663#GCCAAT/1
CAGCCCGCCGATGGCTCCCACAAGNTGTATCTCTGTACAGGTGTTATCGGGAGAATACTTGATTGCATTGGAAAGCAGGTTACTGAAAGCACGTCGGAGCA
+ILLUMINA-3BDE4F_0027:2:1:9216:1663#GCCAAT/1
ZSGDVEMEQHNGGEGGSGJESSWVBO_]_]\DYV\\`SZ_aXaa`ccccccacc[Wcccaaccccacc[cca_cacc_cccaWccc[cccacc[ccb[ca_
@ILLUMINA-3BDE4F_0027:2:1:10636:1724#GCCAAT/1
TGCACAGGTAGCCCCTACGCCGCGNATGAACGACCGGAAACGCCGTCACATGATGGCGAAACCAGCCGACAAACTCCGCTTCCAGCCGCTTCAACACCGCC
+ILLUMINA-3BDE4F_0027:2:1:10636:1724#GCCAAT/1
a^DG_DPGGDQDFFFFQFKD\\Y\Bab]ac[NY[acaXa]cc_ccYccccccbccYcccaccac^acbWccccc_cHabb_aaa[[_Y__a^^Iaab[RY_
@ILLUMINA-3BDE4F_0027:2:1:8882:2205#GCCAAT/1
TCCGAATTAAGCGCATATTCTGGTNCACGACTTTGAAGTGTCGCCCTTTTTAACCTCGCGAAAGCGTTGACTGTTATCCGCCTGAGAAGCCTGGAAGATGA
+ILLUMINA-3BDE4F_0027:2:1:8882:2205#GCCAAT/1
__FH^FSFMK[ELGHHQDVH_]Z[BddddddGaadccY^acfffffffffffffffdfffdfcfffdacef_ffcccfcef`WWW^^W^addeccYccbb^
@ILLUMINA-3BDE4F_0027:2:1:6687:2439#GCCAAT/1
AGATTCGTCGGATATTTGACGTTCNCCAGGTGCACGTAGCCCATCAGGTGGTCGACGTACTCCTTCAGCAC